In [ ]:
from gensim.models import Word2Vec
import pandas as pd
import numpy as np

In [ ]:
recipes = pd.read_parquet('/content/drive/MyDrive/BitirmeData/deduplicated_recipes_recipe1M.parquet', engine='pyarrow')

In [ ]:
recipes

,Name,Instructions,total_time,calories,protein,fat,sugar,ingredients,desc,Concatenated Instructions
0,Yogurt Parfaits,[Layer all ingredients in a serving dish.],NaN,332.06,28.30,8.76,20.82,"[{'name': 'yogurt, greek, plain, nonfat', 'qua...",,Layer all ingredients in a serving dish.
1,"Salt Free, Low Cholesterol Sugar Cookies Recipe","[Cream sugar and butter together till smooth.,...",NaN,3567.77,57.02,175.08,106.92,"[{'name': 'sugars, granulated', 'quantity': 0....",,Cream sugar and butter together till smooth. A...
2,Honey Sriracha Chicken Wings,"[Preheat oven to 400 degrees F., In a large bo...",50.0,3286.10,242.97,225.81,48.16,"[{'name': 'chicken, broilers or fryers, wing, ...",,Preheat oven to 400 degrees F. In a large bowl...
3,Shrimp and Caper Salad,"[In a large bowl, toss the shrimp, green onion...",60.0,3066.25,188.09,251.61,4.95,"[{'name': 'crustaceans, shrimp, raw (not previ...",,"In a large bowl, toss the shrimp, green onions..."
4,Natural Peanut Butter Chocolate Bon Bons,[Measure out the cocoa powder into a mixing bo...,120.0,16252.00,271.44,272.88,3385.56,"[{'name': 'cocoa, dry powder, unsweetened', 'q...",,Measure out the cocoa powder into a mixing bow...
...,...,...,...,...,...,...,...,...,...,...
803217,Italian Sausage Zucchini Soup,"[Slice sausage and brown in the pot., Add cele...",30.0,32.00,1.40,0.34,2.70,"[{'name': 'sweet Italian sausage', 'quantity':...",,Slice sausage and brown in the pot. Add celery...
803218,Soft and Chewy Chocolate Drops,[Microwave unsweetened chocolate and butter 2 ...,89.0,58706.00,131.92,6276.26,713.67,"[{'name': 'unsweetened chocolate square', 'qua...",,Microwave unsweetened chocolate and butter 2 m...
803219,Zucchini Gratin With Feta,"[Preheat oven to 350, Drop zucchini slices int...",37.0,0.00,0.00,0.00,0.00,"[{'name': 'large zucchini', 'quantity': 2.0, '...",,Preheat oven to 350 Drop zucchini slices into ...
803220,"Veal Piccata with Orange, Capers and Tarragon",[Season the veal with salt and pepper and coat...,1.0,6.00,0.60,0.21,0.12,"[{'name': 'veal', 'quantity': 1.0, 'unit': 'po...",,Season the veal with salt and pepper and coat ...


In [ ]:
recipes.describe()

,total_time,calories,protein,fat,sugar
count,568238.000000,8.032220e+05,803222.000000,803222.000000,8.032220e+05
mean,123.923995,2.485197e+03,31.854062,187.112850,1.278212e+02
std,661.072597,1.726076e+04,180.289807,918.738779,1.988720e+03
min,1.000000,0.000000e+00,0.000000,0.000000,0.000000e+00
25%,15.000000,0.000000e+00,0.000000,0.000000,0.000000e+00
50%,35.000000,3.100000e+01,0.200000,0.040000,2.600000e-01
75%,80.000000,6.440000e+02,10.030000,12.230000,2.424000e+01
max,210060.000000,1.345031e+07,99552.000000,402484.550000,1.678980e+06


In [ ]:
recipes.Name.value_counts()

,count
Name,
Yogurt Parfaits,1
Pawpaw Chiffon Pie,1
Merengues Ruta Maya,1
3 Cheese Au Gratin Potatoes With Bacon,1
Light and Refreshing Carrot and Raisin Salad,1
...,...
Donut-Like Deep-Fried Loops or Schuerzkuchen,1
Baked Cheesy Cauliflower,1
Fluffy Easter Cake,1


In [ ]:
recipes_proj = recipes[['Name', 'ingredients', 'Concatenated Instructions']].copy()
recipes_proj.rename({'Name': 'name', 'Concatenated Instructions': 'instructions'}, axis='columns', inplace=True)

In [ ]:
recipes_proj

,name,ingredients,instructions
0,Yogurt Parfaits,"[{'name': 'yogurt, greek, plain, nonfat', 'qua...",Layer all ingredients in a serving dish.
1,"Salt Free, Low Cholesterol Sugar Cookies Recipe","[{'name': 'sugars, granulated', 'quantity': 0....",Cream sugar and butter together till smooth. A...
2,Honey Sriracha Chicken Wings,"[{'name': 'chicken, broilers or fryers, wing, ...",Preheat oven to 400 degrees F. In a large bowl...
3,Shrimp and Caper Salad,"[{'name': 'crustaceans, shrimp, raw (not previ...","In a large bowl, toss the shrimp, green onions..."
4,Natural Peanut Butter Chocolate Bon Bons,"[{'name': 'cocoa, dry powder, unsweetened', 'q...",Measure out the cocoa powder into a mixing bow...
...,...,...,...
803217,Italian Sausage Zucchini Soup,"[{'name': 'sweet Italian sausage', 'quantity':...",Slice sausage and brown in the pot. Add celery...
803218,Soft and Chewy Chocolate Drops,"[{'name': 'unsweetened chocolate square', 'qua...",Microwave unsweetened chocolate and butter 2 m...
803219,Zucchini Gratin With Feta,"[{'name': 'large zucchini', 'quantity': 2.0, '...",Preheat oven to 350 Drop zucchini slices into ...
803220,"Veal Piccata with Orange, Capers and Tarragon","[{'name': 'veal', 'quantity': 1.0, 'unit': 'po...",Season the veal with salt and pepper and coat ...


In [ ]:
data = []
for _, recipe in recipes_proj[:2000].iterrows():
    ing_name = []
    ing_unit = []
    ing_quantity = []
    for ingredient in recipe.ingredients:
        ing_name.append(ingredient['name'])
        ing_unit.append(ingredient['unit'])
        ing_quantity.append(ingredient['quantity'])
    data.append({
        "name": recipe["name"],
        "instructions": recipe["instructions"],
        "ingredient_name": ' '.join(ing_name),
        "ingredient_quantity": ing_quantity,
        "ingredient_unit": ' '.join(ing_unit)
    })

df = pd.DataFrame(data)

del data
del ing_name
del ing_unit
del ing_quantity
del ingredient
del recipe

In [ ]:
df

,name,instructions,ingredient_name,ingredient_quantity,ingredient_unit
0,Yogurt Parfaits,Layer all ingredients in a serving dish.,"yogurt, greek, plain, nonfat strawberries, raw...","[8.0, 1.0, 0.25]",ounce cup cup
1,"Salt Free, Low Cholesterol Sugar Cookies Recipe",Cream sugar and butter together till smooth. A...,"sugars, granulated oil, corn, peanut, and oliv...","[0.5, 0.75, 0.25, 1.0, 0.25, 1.0, 3.5]",cup cup cup teaspoon teaspoon tablespoon cup
2,Honey Sriracha Chicken Wings,Preheat oven to 400 degrees F. In a large bowl...,"chicken, broilers or fryers, wing, meat and sk...","[3.0, 1.0, 0.5, 1.0, 3.0, 0.25, 2.0, 1.0, 1.0,...",pound teaspoon teaspoon tablespoon tablespoon ...
3,Shrimp and Caper Salad,"In a large bowl, toss the shrimp, green onions...","crustaceans, shrimp, raw (not previously froze...","[2.0, 0.75, 1.5, 3.0, 1.0, 0.5, 0.75, 1.0, 1.0]",pound cup cup tablespoon cup cup cup tablespoo...
4,Natural Peanut Butter Chocolate Bon Bons,Measure out the cocoa powder into a mixing bow...,"cocoa, dry powder, unsweetened honey peanut bu...","[12.0, 12.0, 1.0]",cup cup cup
...,...,...,...,...,...
1995,Basic Shortbread,1. Preheat the oven. 2. Cream the margarine an...,"wheat flour, white, all-purpose, unenriched ma...","[150.0, 100.0, 50.0]",g g g
1996,White Breadmaker Bread,Mix ingredients together in order in the bread...,"water, bottled, generic sugars, granulated sal...","[1.25, 1.0, 1.25, 3.5, 1.25]",cup tablespoon teaspoon cup teaspoon
1997,Best Basic White Rice,Place the rice in a large saucepan with the bu...,"rice, white, long-grain, regular, unenriched, ...","[2.0, 1.0, 1.5, 1.0, 4.0]",cup tablespoon teaspoon teaspoon cup
1998,My Best Pinto Beans,Soak the beans in 2 quarts of cold water overn...,"beans, snap, green, raw water, bottled, generi...","[1.0, 2.0, None, 1.0, 14.0]",pound quart ounce teaspoon teaspoon


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Combine text features for TF-IDF
text_data = df["name"] + " " + df["instructions"]

# Fit TF-IDF
tfidf = TfidfVectorizer()
text_vectors = tfidf.fit_transform(text_data)

print(text_vectors.shape)  # Output: (6, <number of unique words>)


(2000, 5736)


In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# One-hot encode ingredient names and units
encoder = OneHotEncoder()
ingredient_name_vectors = encoder.fit_transform(df[["ingredient_name"]])
# ingredient_unit_vectors = encoder.fit_transform(df[["ingredient_unit"]])

# Normalize ingredient quantities
# scaler = StandardScaler()
# ingredient_quantity_vectors = scaler.fit_transform(df[["ingredient_quantity"]])


In [ ]:
def containsNaN(matrix):
  if np.any(np.isnan(matrix.data)):
      print("NaN values found in the data")
      # final_vectors_sparse.data = np.nan_to_num(final_vectors_sparse.data)
  else:
      print("No NaN values found")

In [ ]:
containsNaN(text_vectors)
containsNaN(ingredient_name_vectors)
# containsNaN(ingredient_unit_vectors)
# containsNaN(ingredient_quantity_vectors)

No NaN values found
No NaN values found


In [ ]:
from scipy.sparse import hstack

# Combine all features as sparse matrices
final_vectors_sparse = hstack([
    text_vectors,  # Keep this as sparse
    ingredient_name_vectors  # Already sparse
    # ingredient_unit_vectors  # Already sparse
    # ingredient_quantity_vectors  # Convert to sparse
]).tocsr()

print(final_vectors_sparse.shape)  # This should now work without running out of memory

(2000, 7721)


In [ ]:
if np.any(np.isnan(final_vectors_sparse.data)):
    print("NaN values found in the data")
    final_vectors_sparse.data = np.nan_to_num(final_vectors_sparse.data)
else:
    print("No NaN values found")

No NaN values found


In [ ]:
final_vectors_sparse.data.shape, final_vectors_sparse, final_vectors_sparse.data, final_vectors_sparse[0], final_vectors_sparse.shape

((111654,),
 <2000x7721 sparse matrix of type '<class 'numpy.float64'>'
 	with 111654 stored elements in Compressed Sparse Row format>,
 array([0.40325279, 0.63242708, 0.36900334, ..., 0.12263477, 0.12263477,
        1.        ]),
 <1x7721 sparse matrix of type '<class 'numpy.float64'>'
 	with 9 stored elements in Compressed Sparse Row format>,
 (2000, 7721))

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=1000)
reduced_data = svd.fit_transform(final_vectors_sparse[:2000])

In [ ]:
reduced_data.shape

(2000, 1000)

In [ ]:
reduced_data

array([[ 0.07694897, -0.07599334, -0.05067695, ...,  0.00586954,
         0.02382206, -0.00725869],
       [ 0.32016412, -0.08593134, -0.2587046 , ..., -0.00153484,
         0.00791048,  0.0039595 ],
       [ 0.42753715,  0.08754266,  0.09521266, ..., -0.00205404,
        -0.02136409, -0.0325096 ],
       ...,
       [ 0.4125139 ,  0.03840999,  0.19975719, ..., -0.05763544,
        -0.00709291,  0.00338795],
       [ 0.34789712,  0.02190222,  0.17006382, ...,  0.05471039,
         0.08645946, -0.00587466],
       [ 0.30068306,  0.14442363,  0.10587975, ..., -0.0033345 ,
         0.03011594, -0.01987036]])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
import time

# for i in range(final_vectors_sparse.shape[0]):

start = time.time()
similarity_matrix = cosine_similarity(reduced_data, reduced_data)
print(time.time() - start)

0.040556907653808594


In [ ]:
similarity_matrix.shape, similarity_matrix

((2000, 2000),
 array([[ 1.00000000e+00,  3.35703419e-03,  2.77590043e-02, ...,
         -6.63779220e-04,  1.21535329e-02,  1.09588436e-02],
        [ 3.35703419e-03,  1.00000000e+00,  6.68747627e-02, ...,
          4.09251067e-02,  4.10877784e-02,  9.45652425e-03],
        [ 2.77590043e-02,  6.68747627e-02,  1.00000000e+00, ...,
          1.71466591e-01,  1.22229435e-01,  1.12776800e-01],
        ...,
        [-6.63779220e-04,  4.09251067e-02,  1.71466591e-01, ...,
          1.00000000e+00,  1.56358001e-01,  7.75590062e-02],
        [ 1.21535329e-02,  4.10877784e-02,  1.22229435e-01, ...,
          1.56358001e-01,  1.00000000e+00,  1.62045617e-01],
        [ 1.09588436e-02,  9.45652425e-03,  1.12776800e-01, ...,
          7.75590062e-02,  1.62045617e-01,  1.00000000e+00]]))

In [ ]:
cosine_sim_matrix = similarity_matrix

np.fill_diagonal(cosine_sim_matrix, -np.inf)

# Her tarif için en yakın 10 tarifin indekslerini ve similarity değerlerini bulalım
top_10_similarities = []
top_10_indices = []

for i in range(cosine_sim_matrix.shape[0]):
    # İlgili satırdaki en yüksek 10 değeri ve indeksleri al
    indices = np.argpartition(cosine_sim_matrix[i], -10)[-10:]
    values = cosine_sim_matrix[i][indices]
    # Değerleri büyükten küçüğe sıralayın
    sorted_indices = indices[np.argsort(-values)]
    sorted_values = values[np.argsort(-values)]

    top_10_indices.append(sorted_indices)
    top_10_similarities.append(sorted_values)

# Sonuçları göstermek için örnek
for i in range(5):  # İlk 5 tarif için
    print(f"Tarif {i} için en yakın 10 tarif:")
    for j in range(10):
        print(f"  - Tarif {top_10_indices[i][j]} (benzerlik: {top_10_similarities[i][j]:.2f})")

Tarif 0 için en yakın 10 tarif:
  - Tarif 1956 (benzerlik: 0.26)
  - Tarif 1887 (benzerlik: 0.24)
  - Tarif 1416 (benzerlik: 0.23)
  - Tarif 1389 (benzerlik: 0.21)
  - Tarif 1221 (benzerlik: 0.21)
  - Tarif 893 (benzerlik: 0.20)
  - Tarif 295 (benzerlik: 0.18)
  - Tarif 515 (benzerlik: 0.18)
  - Tarif 374 (benzerlik: 0.16)
  - Tarif 315 (benzerlik: 0.15)
Tarif 1 için en yakın 10 tarif:
  - Tarif 1906 (benzerlik: 0.31)
  - Tarif 1864 (benzerlik: 0.28)
  - Tarif 958 (benzerlik: 0.27)
  - Tarif 648 (benzerlik: 0.25)
  - Tarif 1679 (benzerlik: 0.23)
  - Tarif 1897 (benzerlik: 0.23)
  - Tarif 1565 (benzerlik: 0.22)
  - Tarif 1788 (benzerlik: 0.22)
  - Tarif 1120 (benzerlik: 0.22)
  - Tarif 184 (benzerlik: 0.21)
Tarif 2 için en yakın 10 tarif:
  - Tarif 684 (benzerlik: 0.57)
  - Tarif 1068 (benzerlik: 0.55)
  - Tarif 1935 (benzerlik: 0.51)
  - Tarif 1718 (benzerlik: 0.51)
  - Tarif 680 (benzerlik: 0.43)
  - Tarif 390 (benzerlik: 0.42)
  - Tarif 1587 (benzerlik: 0.37)
  - Tarif 1783 (benzerli

In [ ]:
cosine_sim_matrix = similarity_matrix

np.fill_diagonal(cosine_sim_matrix, -np.inf)


# Sapmalar için parametreler
num_closest = 10  # En yakın kaç tarif seçilecek
num_outliers = 2  # Kaç sapma eklenecek
total_recipes = cosine_sim_matrix.shape[0]  # Toplam tarif sayısı

# Tüm tarifler için listeyi oluştur
final_lists = []

for i in range(total_recipes):
    # En yakın tarifleri al
    indices = np.argpartition(cosine_sim_matrix[i], -num_closest)[-num_closest:]
    values = cosine_sim_matrix[i][indices]
    sorted_indices = indices[np.argsort(-values)]

    # Rastgele sapma ekleme
    random_outliers = np.random.choice(
        [idx for idx in range(total_recipes) if idx not in sorted_indices and idx != i],
        size=num_outliers,
        replace=False
    )

    # Sapmalar ve yakın tarifleri birleştir
    final_list = list(sorted_indices) + list(random_outliers)
    # np.random.shuffle(final_list)  # Listeyi karıştırarak rastgelelik ekle

    final_lists.append(final_list)

# Sonuçları gösterme
for i in range(5):  # İlk 5 tarif için örnek
    print(f"Tarif {i} için öneri listesi (sapmalar dahil):")
    print(final_lists[i])
    for j in final_lists[i]:
      print(cosine_sim_matrix[i, j], end=' ')
    print()

Tarif 0 için öneri listesi (sapmalar dahil):
[1956, 1887, 1416, 1389, 1221, 893, 295, 515, 374, 315, 806, 762]
0.25862366478088794 0.2355588227758807 0.23085909079130487 0.20991892654436312 0.2097637686743014 0.20075916333913052 0.17648885268882375 0.17580863100264038 0.1649070051504987 0.14903901189832378 0.00594839765015176 0.013956237247977982 
Tarif 1 için öneri listesi (sapmalar dahil):
[1906, 1864, 958, 648, 1679, 1897, 1565, 1788, 1120, 184, 995, 1451]
0.31325895220906663 0.27869399806777945 0.2706772482032135 0.2466049743786018 0.22791172492581516 0.2270615187089254 0.22266792983946998 0.22100613899649166 0.22087214002835537 0.21219647883634038 0.057685815353490315 0.03427392157535951 
Tarif 2 için öneri listesi (sapmalar dahil):
[684, 1068, 1935, 1718, 680, 390, 1587, 1783, 277, 1137, 226, 366]
0.5677649650824561 0.5536672146721645 0.5115909001750577 0.505602989731857 0.42519641332696956 0.42491184449510155 0.3697488226686928 0.27979265963528877 0.27790688859990625 0.269133190

In [ ]:
user_preferences = final_lists[:10]

In [ ]:
# Toplam kullanıcı ve tarif sayıları
num_users = len(user_preferences)
num_items = max(max(prefs) for prefs in user_preferences) + 1  # İndeksler 0'dan başladığı için +1 eklenir

# Seyrek matrisin verilerini depolamak için listeler
row_indices = []
col_indices = []
data = []

# Kullanıcı tercihlerini matris formatına dönüştürme
for user_id, items in enumerate(user_preferences):
    for item_id in items:
        row_indices.append(user_id)
        col_indices.append(item_id);
        data.append(1)  # Tercih edilen tarifler için 1 değeri atanır

# CSR formatında seyrek matris oluşturma
user_item_matrix = csr_matrix((data, (row_indices, col_indices)), shape=(num_users, num_items))

user_item_matrix = np.asarray(user_item_matrix.todense())
user_item_matrix

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
from sklearn import svm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
# Öğeler arası kosinüs benzerlik matrisi
item_similarity_matrix = cosine_similarity(user_item_matrix.T)

# Kullanıcıların beğendiği öğeleri temel alarak her öğe için özellik vektörleri oluşturma
def create_feature_matrix(user_item_matrix, item_similarity_matrix):
    num_users, num_items = user_item_matrix.shape
    feature_matrix = np.zeros((num_users, num_items))
    for user in range(num_users):
        for item in range(num_items):
            # Kullanıcının beğendiği öğelerle mevcut öğe arasındaki benzerliklerin ortalamasını al
            liked_items = np.where(user_item_matrix[user] == 1)[0]
            if len(liked_items) > 0:
                feature_matrix[user, item] = item_similarity_matrix[item, liked_items].mean()
    return feature_matrix

# Özellik matrisi oluşturma
X = create_feature_matrix(user_item_matrix, item_similarity_matrix)

# Etiket matrisi (kullanıcıların beğendiği öğeler)
y = user_item_matrix

In [ ]:
# Her kullanıcı için SVM modelini eğitme ve öneri yapma
for user in range(user_item_matrix.shape[0]):
    # Eğitim ve test verilerini ayırma
    X_train, X_test, y_train, y_test = train_test_split(
        X[user].reshape(-1, 1), y[user], test_size=0.2, random_state=42
    )

    # SVM modelini oluşturma ve eğitme
    model = svm.SVC(kernel='linear')
    model.fit(X_train, y_train)

    # Tahmin yapma
    predictions = model.predict(X_test)

    # Model doğruluğunu değerlendirme
    accuracy = accuracy_score(y_test, predictions)
    print(f"Kullanıcı {user + 1} için model doğruluğu: {accuracy:.2f}")

    # Henüz beğenmediği öğeler için tahmin yapma
    unseen_items = np.where(user_item_matrix[user] == 0)[0]
    if len(unseen_items) > 0:
        scores = model.decision_function(X[user, unseen_items].reshape(-1, 1))
        recommended_item = unseen_items[np.argmax(scores)]
        print(f"Kullanıcı {user + 1} için önerilen öğe: {recommended_item + 1}")
        print(df.iloc[user], '\n', df.iloc[recommended_item], '\n')
    else:
        print(f"Kullanıcı {user + 1} için önerilecek yeni öğe yok.\n")

Kullanıcı 1 için model doğruluğu: 1.00
Kullanıcı 1 için önerilen öğe: 1
name                                                     Yogurt Parfaits
instructions                    Layer all ingredients in a serving dish.
ingredient_name        yogurt, greek, plain, nonfat strawberries, raw...
ingredient_quantity                                     [8.0, 1.0, 0.25]
ingredient_unit                                            ounce cup cup
Name: 0, dtype: object 
 name                                                     Yogurt Parfaits
instructions                    Layer all ingredients in a serving dish.
ingredient_name        yogurt, greek, plain, nonfat strawberries, raw...
ingredient_quantity                                     [8.0, 1.0, 0.25]
ingredient_unit                                            ounce cup cup
Name: 0, dtype: object 

Kullanıcı 2 için model doğruluğu: 1.00
Kullanıcı 2 için önerilen öğe: 423
name                     Salt Free, Low Cholesterol Sugar Cookies Recipe
i

In [ ]:
final_lists[:2]

[[1956, 1887, 1416, 1389, 1221, 893, 295, 515, 374, 315, 806, 762],
 [1906, 1864, 958, 648, 1679, 1897, 1565, 1788, 1120, 184, 995, 1451]]

In [ ]:
train_set = reduced_data[final_lists[:9]]

In [ ]:
# SVM modelini oluşturma ve eğitme
model_reg = svm.SVR(kernel='linear')
model_reg.fit(train_set[0], np.ones(train_set[0].shape[0]))

# Tahmin yapma
predictions = model_reg.predict(reduced_data[final_lists[9]])
predictions

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [ ]:
user_preferences_recipes = []
for i in range(10):
  user_preferences_recipes.append(df.iloc[final_lists[i]])

In [ ]:
df.iloc[2]

,2
name,Honey Sriracha Chicken Wings
instructions,Preheat oven to 400 degrees F. In a large bowl...
ingredient_name,"chicken, broilers or fryers, wing, meat and sk..."
ingredient_quantity,"[3.0, 1.0, 0.5, 1.0, 3.0, 0.25, 2.0, 1.0, 1.0,..."
ingredient_unit,pound teaspoon teaspoon tablespoon tablespoon ...


In [ ]:
user_preferences_recipes[2]

,name,instructions,ingredient_name,ingredient_quantity,ingredient_unit
684,Jerry's Wings,"In a large skillet or deep fryer, heat oil to ...","oil, olive, salad or cooking chicken, broilers...","[1.0, 1.0, 0.75, 0.25]",cup pound cup cup
1068,Easy Chinese Chicken Wings,Whisk all ingredients (except wings) together ...,"chicken, broilers or fryers, wing, meat and sk...","[4.0, 1.0, 1.0, 2.0, 1.0, 0.25]",pound cup cup cup teaspoon teaspoon
1935,Sweet & Sour Glaze Chicken Wings,Rinse and drain chicken wings. Combine remaini...,"chicken, broilers or fryers, wing, meat and sk...","[3.0, 12.0, 12.0, 14.0, 3.0, 2.0, 2.0]",pound cup cup cup tablespoon tablespoon tables...
1718,Sticky Honey Garlic Chicken Wings,Preheat oven to 425F. Place wings on a greased...,"chicken, broilers or fryers, wing, meat and sk...","[3.0, 13.0, 14.0, 3.0, 2.0, 1.0, 12.0]",pound cup cup tablespoon tablespoon teaspoon t...
680,Spicy Apricot Wings,Light a gas grill. Wrap the wood chips in heav...,"apricots, dried, sulfured, uncooked apricots, ...","[0.5, 0.5, 2.0, 2.0, 1.0, 1.0, 1.0, 2.0, 1.0, ...",cup cup tablespoon tablespoon tablespoon table...
390,"Spicy Hot Wings, Low Carb",Place wings in a greased 15x10x1 inch baking p...,"chicken, broilers or fryers, wing, meat and sk...","[2.0, 12.0, None, 34.0, 14.0]",pound cup tablespoon teaspoon teaspoon
1587,Peri Peri Wings,Mix dry rub ingredients well and season wings ...,"chicken, broilers or fryers, wing, meat and sk...","[2.0, 1.0, 1.0, 2.0, 1.0, 1.0, 0.5, 0.5, 0.5]",pound tablespoon tablespoon teaspoon teaspoon ...
1783,Sriracha Garlic Salt,Line a baking sheet with parchment paper. Mix ...,"salt, table sauce, hot chile, sriracha spices,...","[12.0, 5.0, 1.0]",cup teaspoon teaspoon
277,Grilled Chicken Wings With Tandoori Flavorings,Heat oil in an 8-inch skillet over low heat. M...,"oil, olive, salad or cooking spices, cumin see...","[14.0, 3.0, 1.0, 2.0, 2.0, 1.0, 12.0, 13.0, 1....",cup tablespoon tablespoon teaspoon teaspoon te...
1137,Barbecue Spice Rub for Wings,Combine all ingredients and mix thoroughly. St...,"sugars, brown spices, chili powder spices, pap...","[1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, ...",cup tablespoon tablespoon tablespoon teaspoon ...
